In [1]:
import pandas as pd
import os
import csv
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from utils import data_preparation

In [2]:
os.getcwd()
repo = '/home/yiyi/Documents/masterthesis/CPD'

In [3]:
data_dir = os.path.join(repo, 'data')
file ='cleand_query_output_csv/019b8823-aa09-4d8f-8b8d-26433d76dff3'

In [4]:
handler = data_preparation.TextHandler(os.path.join(data_dir, file))

In [ ]:
handler.prepare()

In [6]:
len(handler.data)

56914

In [7]:
handler.data[:100]

['worth money',
 'half closing door',
 'light rain sound tv due satelite interfernece',
 'small pool',
 'strip house restaurant small menu',
 'view le',
 'staff friendly',
 'activity site rip',
 'fantastic time',
 'excellent resort',
 'aj man booking snorkling trip stacey also let u borrow car pick beverage service amazing',
 'day loved every second',
 'shared amenity casa marina next door real nice bonus',
 'best place catch sunset malory square roof reach',
 'probably best stay ever resort',
 'despite two night could relax enjoy key west reach resort home base',
 'hotel block away quiet end duval street parked car walked around everything wanted see',
 'little pool patch beach quite young family',
 'chaotic kindergarten room top floor slice view ocean nice since priced view',
 'high ceiling may u lucky though think room clean neat someone mentioned whole place recently renovated new feel',
 'bed amazingly good',
 'oddly lot noise one afternoon due wedding party second floor ballroom 

In [8]:
# generate BERT data
pretrained = 'roberta-base-nli-stsb-mean-tokens'
training_bert = data_preparation.bert_embeddings_from_list(handler.data, pretrained)

In [ ]:
bert_base = 'bert-base-nli-stsb-mean-tokens'
training_bert = data_preparation.bert_embeddings_from_list(handler.data, bert_base)

In [9]:
training_bert.shape

(56914, 768)

In [10]:
from contextualized_topic_models.models.ctm import CTM
from contextualized_topic_models.datasets.dataset import CTMDataset

In [25]:
training_dataset = CTMDataset(handler.bow, training_bert, handler.idx2token)
ctm = CTM(input_size=len(handler.vocab), bert_input_size=768, lr=0.01, inference_type="contextual", n_components=20)

ctm.fit(training_dataset)

Settings: 
               N Components: 20
               Topic Prior Mean: 0.0
               Topic Prior Variance: 0.95
               Model Type: prodLDA
               Hidden Sizes: (100, 100)
               Activation: softplus
               Dropout: 0.2
               Learn Priors: True
               Learning Rate: 0.01
               Momentum: 0.99
               Reduce On Plateau: False
               Save Dir: None
Epoch: [1/100]	Samples: [56914/5691400]	Train Loss: 74.97642041207801	Time: 0:00:05.613520
Epoch: [2/100]	Samples: [113828/5691400]	Train Loss: 72.3603455183959	Time: 0:00:05.508517
Epoch: [3/100]	Samples: [170742/5691400]	Train Loss: 71.78372893202902	Time: 0:00:05.496876
Epoch: [4/100]	Samples: [227656/5691400]	Train Loss: 71.48978090633112	Time: 0:00:05.519740
Epoch: [5/100]	Samples: [284570/5691400]	Train Loss: 71.33326870417291	Time: 0:00:05.514778
Epoch: [6/100]	Samples: [341484/5691400]	Train Loss: 71.26361088917461	Time: 0:00:05.504001
Epoch: [7/100]	Sampl

In [ ]:


training_dataset = CTMDataset(handler.bow, training_bert, handler.idx2token)

ctm = CTM(input_size=len(handler.vocab), bert_input_size=768, inference_type="combined", n_components=20)

ctm.fit(training_dataset) # run the model

In [26]:
ctm.get_topic_lists(5)


[['pleasant', 'accommodating', 'friendly', 'attentive', 'professional'],
 ['casa', 'marina', 'sister', 'better', 'nicer'],
 ['key', 'west', 'best', 'visit', 'place'],
 ['waldorf', 'astoria', 'hilton', 'name', 'expect'],
 ['towel', 'ice', 'set', 'fresh', 'afternoon'],
 ['incredible', 'fabulous', 'ideal', 'atmosphere', 'absolutely'],
 ['street', 'close', 'walking', 'quiet', 'easy'],
 ['negative', 'read', 'reading', 'management', 'concern'],
 ['highly', 'somewhere', 'anyone', 'definitely', 'elsewhere'],
 ['fee', 'charge', 'per', 'parking', 'park'],
 ['weekend', 'week', 'wife', 'wedding', 'husband'],
 ['hotel', 'nice', 'good', 'service', 'great'],
 ['food', 'buffet', 'breakfast', 'strip', 'house'],
 ['comfortable', 'modern', 'nicely', 'tv', 'machine'],
 ['called', 'said', 'call', 'ready', 'told'],
 ['make', 'sure', 'tour', 'care', 'made'],
 ['dirty', 'elevator', 'loud', 'air', 'wall'],
 ['become', 'waste', 'whilst', 'according', 'familiar'],
 ['sand', 'pier', 'private', 'swimming', 'sandy'

In [19]:
from contextualized_topic_models.evaluation.measures import TopicDiversity, CoherenceNPMI,\
    CoherenceWordEmbeddings, InvertedRBO


td = TopicDiversity(ctm.get_topic_lists(25))
td.score(topk=25)

0.918

In [27]:

rbo = InvertedRBO(ctm.get_topic_lists(10))
rbo.score()

0.9993871527758647

In [28]:
texts = [doc.split() for doc in handler.data]

In [29]:
npmi = CoherenceNPMI(texts =texts, topics= ctm.get_topic_lists(10))

In [30]:
npmi.score()

0.07307711813893471